In [ ]:
%pip install awswrangler pandas geopandas mapclassify folium boto3 -q

In [ ]:
import awswrangler as wr
import pandas as pd
from datetime import datetime
import boto3
import json

In [ ]:
# create a data source for athena via a glue data catalog to overture:
# https://docs.overturemaps.org/getting-data/athena-aws/
client = boto3.client('athena', region_name='us-west-2')
client.create_data_catalog(
    Name='overture_data_catalog', 
    Type='GLUE',
    Parameters={'catalog-id': '913550007193'},
)

In [ ]:
# if required this policy will need to be added to the IAM user for Athena queries.
iam_client = boto3.client('iam')
iam_client.create_policy(
    PolicyName='OvertureGlueCatalogAccess',
    PolicyDocument=json.dumps(
        { 
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                         "glue:GetTable*",
                          "glue:GetDatabase*",
                          "glue:GetPartition*"
                    ],
                    "Resource": [
                        "arn:aws:glue:us-west-2:913550007193:catalog",
                        "arn:aws:glue:us-west-2:913550007193:database/release",
                        "arn:aws:glue:us-west-2:913550007193:table/*"
                    ]
                }
            ]
        }
    )
)

In [ ]:
# list out primary categories from places data
df = wr.athena.read_sql_query("""
SELECT distinct categories.primary FROM overture.release.v2024_12_18_0 WHERE type='place' order by categories.primary asc
""", database="release", data_source="overture_data_catalog", ctas_approach=False)
df

In [ ]:
# bus stop data as an example is in infrastructure
# query via athena into dataframe - at this point it can be exported as CSV/JSON etc.
df = wr.athena.read_sql_query("""
SELECT   
    names,
    subtype,
    class,
    ST_GEOMFROMBINARY(geometry) AS geometry
FROM
    overture.release.v2024_12_18_0
WHERE type='infrastructure' 
    AND bbox.xmin BETWEEN 140.952 AND 155.645
    AND bbox.ymin BETWEEN -37.667 AND -28.143 
    AND class = 'bus_stop'
""", database="release", data_source="overture_data_catalog", ctas_approach=False)

In [ ]:
# create a spatial dataframe for visualisation, importatant to set the projection.
import geopandas
gdf = geopandas.GeoDataFrame(df)
gdf = gdf.set_crs('EPSG:4326', allow_override=True)

In [ ]:
import folium
m = gdf.explore()
folium.TileLayer("CartoDB positron", show=False).add_to(m) 
m  # show map

In [ ]:
# this example get everything labels as a park.
park_df = wr.athena.read_sql_query("""
SELECT 
    names,
    class,
    surface,
    source_tags,
    ST_GEOMFROMBINARY(geometry) AS geometry
FROM
    overture.release.v2024_12_18_0
WHERE type='land_use' 
    AND bbox.xmin BETWEEN 140.256 AND 155.645
    AND bbox.ymin BETWEEN -37.667 AND -28.638 
    AND subtype = 'park'
""", database="release", data_source="overture_data_catalog", ctas_approach=False)

In [ ]:
park_gdf = geopandas.GeoDataFrame(park_df)
park_gdf = park_gdf.set_crs('EPSG:4326', allow_override=True)

In [ ]:
import folium

m = park_gdf.explore()
folium.TileLayer("CartoDB positron", show=False).add_to(m) 
m  # show map